In [ ]:
import requests
import json
import pandas as pd
from google.colab import files

print("Upload Your File ")

uploaded = files.upload()

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

print("✔")
print(df.head())

📌 قم الآن برفع ملف: YELP_WITH_Sentiment Analysis.csv


Saving YELP_WITH_Sentiment Analysis.csv to YELP_WITH_Sentiment Analysis.csv
✔ الملف تم تحميله بنجاح!
              business_id        date               review_id  stars  \
0  9yKzy9PApeiPPOUJEtnvkg  2011-01-26  fWKvX83p0-ka4JS3dc6E5A      5   
1  ZRJwVLyzEJq1VAihDhYiow  2011-07-27  IjZ33sJrzXqU-0X6U8NwyA      5   
2  6oRAC4uyJCsJl1X0WZpVSA  2012-06-14  IESLBzqUCLdSzSqm0eCSxQ      4   
3  _1QQZuf4zZOyFCvXc0o6Vg  2010-05-27  G-WvGaISbqqaMHlNnByodA      5   
4  6ozycU1RpktNG2-1BroVtw  2012-01-05  1uJFq2r5QfJG_6ExMRCaGw      5   

                                                text    type  \
0  wife take birthday breakfast excellent    weat...  review   
1  idea people bad review place go probably grip ...  review   
2      love gyro plate rice good dig candy selection  review   
3  rosie dakota love chaparral dog park convenien...  review   
4  general manager scott petello good egg detail ...  review   

                  user_id  cool  useful  funny sentiment  sentiment_score  
0  rL

In [ ]:
API_KEY = "*******"

sample = df.iloc[15]

review_text = sample["text"]
sentiment = sample["sentiment"]
sentiment_score = sample["sentiment_score"]

print("\n----------------------------")
print("📌 Review Loaded:")
print(review_text)
print("\nSentiment:", sentiment)
print("Sentiment Score:", sentiment_score)
print("----------------------------")



----------------------------
📌 Review Loaded:
worth    salad small pizza absolutely bad service maybe guy grandma die know want tell mad experience order small pizza salad guy care take    sat look hmm sign say x large pizza large salad    wow nice guy told leave hungry mad unsatisfied     owner teach employee value upselle tell special small affect customer experience negatively     salad severely overpriced     desperate

Sentiment: negative
Sentiment Score: 0.9009035229682922
----------------------------


In [ ]:
payload1 = {
    "model": "openai/gpt-oss-20b:free",
    "reasoning": {"enabled": True},
    "messages": [
        {
            "role": "user",
            "content": f"""
هذه مراجعة Yelp:

{review_text}

تحليل المشاعر: {sentiment}
درجة المشاعر: {sentiment_score}

يرجى تحليل المراجعة التالية بعناية:
- لخّص كل نقطة رئيسية بترتيب منطقي.
- حافظ على الوضوح، ولا تخلط الكلمات أو الجمل.
- أخرج النتائج في أقسام منظمة: الخدمة، السعر، دقة الطلب، البيع الإضافي (Upselling).
"""
        }
    ]
}

resp1 = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    },
    data=json.dumps(payload1)
)

msg1 = resp1.json()["choices"][0]["message"]

assistant_answer_1 = msg1.get("content")
assistant_reasoning = msg1.get("reasoning_details")  T

print("\n=== 📌 OUTPUT FROM CALL 1 ===")
print(assistant_answer_1)


=== 📌 OUTPUT FROM CALL 1 ===
**الخدمة**  
- العملاء يذكرون أن الخدمة “بشكلٍ أسوأ”؛ الموظفون غير مهتمين، ولا يعتنون بطلبات الزبائن.  
- تشير الألفاظ المتكررة إلى انزعاج واضح وصراخ أو احتجاج، ما يضعف تجربة المطعم بشكل ملحوظ.  
- حتى مع وجود أحد الأوصال (المُشرف؟)، لا يبدو أن هناك تدخلًا فعالًا للحد من المشكلة.

**السعر**  
- يُعتبر السلمون (أو السلطة) “غاليًا جدًا” بالنسبة للنوع والحجم المقدم.  
- الاستياء ينشأ من عدم تحقيق قيمة مقابل المال، خاصةً عند مقارنة الحجم المنشود مع الحجم الفعلي.

**دقة الطلب**  
- الطلب الأصلي هو “بيتزا صغيرة وسلطة صغيرة”، لكن المُقدّم يُشير إلى “بيتزا كبيرة” أو “سلطة كبيرة”.  
- اللاطابق بين الطلب والسلعة المسلمة يبرز خطأ واضح في تنفيذ الطلب، ما يزيد الطين بلة على الخدمة.

**البيع الإضافي (Upselling)**  
- يوجد ذكر واضح لوجود سياسة “البيع الإضافي” أو “التسويق للمنتجات الصغيرة”.  
- المستفيد من هذه العملية يصفها بأنها “تؤثر سلبًا على تجربة العميل”، ما يعني أنها تُعتبر إكراهًا أو إزعاجًا للزبائن.  
- النتيجة هي أن العميل يشعر بالتفكير السلبي تجاه المطعم بسبب هذ

In [ ]:
messages = [
    {
        "role": "user",
        "content": f"""
هنا نفس مراجعة Yelp:

{review_text}

تحليل المشاعر: {sentiment}
درجة المشاعر: {sentiment_score}
"""
    },
    {
        "role": "assistant",
        "content": assistant_answer_1,
        "reasoning_details": assistant_reasoning
    },
    {
        "role": "user",
        "content": "ما أهم توصية عملية يمكن للمطعم تطبيقها اعتماداً على تفكيرك الداخلي؟"
    }
]

payload2 = {
    "model": "openai/gpt-oss-20b:free",
    "reasoning": {"enabled": True},
    "messages": messages
}

resp2 = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    },
    data=json.dumps(payload2)
)

final_output = resp2.json()["choices"][0]["message"]["content"]

print("\n=== 📌 FINAL OUTPUT (CALL 2) ===")
print(final_output)


=== 📌 FINAL OUTPUT (CALL 2) ===
**التوصية الأكثر فاعلية التي يمكن أن يطبقها المطعم**

> **إنشاء عملية تحقق شاملة للطلبات (Order‑Verification Flow) وتدريب الموظفين عليها مع إلغاء تطبيق استراتيجيات البيع الإضافي القسرية.**

---

### لماذا هذه الخطوة؟  

| المشكلة | لماذا يؤثر الأمر على العميل؟ | كيف تحل هذه الخطوة المشكلة؟ |
|---------|------------------------------|------------------------------|
| **طابع الخدمة غير المريح** | يُشعر العميل بأنه لا يُحترم أو يُسمع، ما يسبب استياءً واضحاً. | يرفع مستوى الوعي لدى الموظفين من خلال التدريب على أخلاق التعامل، ويضمن مراجعة الطلب قبل تنفيذها، ما يقلل الأخطاء وتحسن الشعور بالمحبة. |
| **خطأ في الحجم/الكمية** | التوقعات المرتفعة لا تُلبَّى؛ من المرجح أن يشعر الزبون بالغش أو عدم النِّزاه. | مع عملية التحقق، تُعاد صياغة الطلب (الصيغة، الحجم، المكونات) مسبقاً مع العميل أو على شاشة واضحة؛ مما يمنع الأخطاء. |
| **البيع الإضافي القسري** | يُعتبر إكراهًا ويُضر بالثقة بين العميل والمطعم، ويسبب توترًا. | بإلغاء أو تقييد الاستراتيجيات المُكافلة التي تقوّض